<left>FINM 33150 - Quantitative Trading Strategies</left>
<left>Winter 2023</left>
<br>
<h1><center> Homework 5: FX Carry Strategy </center></h1>
<center>Due - 23:00 [CST] February 11th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

<h5> Imports </h5>

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import quandl

<h5> Constants </h5>

In [3]:
end_date = datetime.now().date().strftime('%Y-%m-%d')
quarterly_coupon = 0.25
weekly_interval = 1/52
float_years = 5
weekly_investment_USD = 10**7

<h5> Helper Functions </h5>

In [4]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
def grab_quandl_table(table_path,start_date = None,end_date = None):
    data = quandl.get(table_path, start_date=start_date, end_date=end_date, returns="pandas",
                      api_key='JbMPn9bSpFPNS7Z7PcZy')
    return data

In [5]:
def sort_tags(x):
    temp_l = x.split('-')
    if temp_l[1] == 'Year':
        return int(temp_l[0])
    elif temp_l[1] == 'Month':
        return int(temp_l[0])/12
    elif temp_l[1] == 'Week':
        return int(temp_l[0])/52
    else:
        raise Exception("Unknown Tenure")

In [6]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
def compute_zcb_curve(spot_rates, coupon_frequency):
    zcb_curve = spot_rates.copy()
    tenors = zcb_curve.columns.map(sort_tags).values
    for tenor_tag in zcb_curve.columns:
        tenor = sort_tags(tenor_tag)
        spot_rates = zcb_curve[tenor_tag]
        coupon = spot_rates * coupon_frequency
        times = np.arange(tenor - coupon_frequency, 0, step = -coupon_frequency)[::-1]
        # Linear interpolation
        preceding_coupons_val = zcb_curve.apply(lambda x: np.exp(-np.interp(times, tenors, x.values)*times).sum(),
                                                axis = 1)
        preceding_coupons_val = preceding_coupons_val * coupon
        zcb_curve[tenor_tag] = -np.log((1 - preceding_coupons_val)/(1+coupon))/tenor
    return zcb_curve

In [7]:
"""
This code was given by Dr. Boonstra, B., Ph.D. for
University of Chicago FINM 33150 Quandl Options Data Fetching guidelines
"""
def zcb_bond_price(zcb_curve, coupon_rates, tenure, coupon_frequency, offset = 0):
    df = zcb_curve.copy()
    times = np.arange(tenure, 0, step=-coupon_frequency)[::-1]
    times = times - offset # offset if time to maturity is not exactly the tenor of the bond
    tenures = df.columns.map(lambda x: int(x.split('-')[0])).values
    if times.shape[0]==0:
        prices = pd.DataFrame(index=zcb_curve.index, columns=['data']).fillna(1.0)
    else:
        # Linear interpolation
        coupons = df.apply(lambda x: np.exp(-np.interp(times, tenures, x.values)*times).sum(), axis = 1)
        r_T = df.apply(lambda x: np.interp(times[-1], tenures, x.values), axis = 1)
        prices = np.exp(-tenure*r_T) + coupon_frequency * coupon_rates * coupons
    return prices

In [8]:
def FX_Fixed_Carry(OIS, FX, cap):
    df = FX.merge(OIS, how = 'inner', left_index = True, right_index = True).copy().dropna()
    df = df[df.index.map(lambda x: x.weekday()) == 2] # Wednesday-fy
    f, ir = df.columns
    # initial FX
    df[('Entry ' + f + ' FX')] = df[f]
    # terminal FX
    df[('Exit ' + f + ' FX')] = df[f].shift(-1)
    # initial IR
    df[('Fixed IR (' + f + ')')] = df[ir]
    # initial borrow (5x leverage)
    ret_series = -(cap*4/5)*df[f]
    df[('Entry Borrow (' + f + ')')] = ret_series
    # 1-week interest in borrowed currency (OIS + 50 b.p.)
    ret_series = ret_series * np.exp((df[ir] + 50*10**(-4))/360 * 7)
    df[('Exit Borrow (' + f + ')')] = ret_series
    # back to funding-currency
    ret_series = ret_series / df[f].shift(-1) #convert back to USD
    df['Fixed 1-week Interest (USD)'] =  ret_series + (cap*4/5)
    return df.iloc[:, 2:]

In [9]:
def FX_Float_Carry(YC, FX, cap, tenure, coupon_frequency, interval):
    df = FX.merge(YC, how = 'inner', left_index = True, right_index = True).copy().dropna()
    df = df[df.index.map(lambda x: x.weekday()) == 2] # Wednesday-fy
    f = df.columns[0] #column name
    tenure_tag = str(tenure) + '-Year'
    # computing zcb curve
    zcb_df = compute_zcb_curve(df.iloc[:, 1:], coupon_frequency)
    # computing entry and exit bond prices
    entry_bond = zcb_bond_price(zcb_df, df[tenure_tag], tenure, coupon_frequency)
    exit_bond = zcb_bond_price(zcb_df, df[tenure_tag], tenure, coupon_frequency, offset = interval)
    # initial FX
    df[('Entry ' + f + ' FX')] = df[f]
    # terminal FX
    df[('Exit ' + f + ' FX')] = df[f].shift(-1)
    # initial IR
    df[('Entry IR (' + f + ')')] = df[tenure_tag]
    # terminal IR
    df[('Exit IR (' + f + ')')] = df[tenure_tag].shift(-1)
    # simulating trade
    ret_series = cap * df[f] # exchange to home currency
    ret_series = ret_series / entry_bond # buy long-term bonds
    df[('Entry Lend (' + f + ')')] = ret_series
    ret_series = ret_series * exit_bond.shift(-1) # sell long-term bonds
    df[('Exit Lend (' + f + ')')] = ret_series
    ret_series = ret_series/df[f].shift(-1) # convert back to USD
    df['Float 1-week Interest (USD)'] = ret_series - cap
    # reorganizing columns
    cols = df.columns.tolist()[-7:]
    return df[cols]

In [10]:
def FX_Fixed_Float_Carry(OIS, FX_fixed, YC, FX_float, cap, tenure, coupon_frequency, interval):
    # computing FX Fixed Carry
    fixed_df = FX_Fixed_Carry(OIS, FX_fixed, cap)
    # computing FX Float Carry
    float_df = FX_Float_Carry(YC, FX_float, cap, tenure, coupon_frequency, interval)
    # merging data
    df = fixed_df.merge(float_df, how = 'inner', left_index = True, right_index = True)
    # setting entry date as columns
    df.reset_index(inplace = True)
    df.rename(columns={'index':'Entry Date'}, inplace = True)
    # creating exit date
    df['Exit Date'] = df['Entry Date'].shift(-1)
    # reorganizing columns
    cols = df.columns.tolist()
    cols = cols[:1] + cols[-1:] + cols[1:-1]
    df = df[cols]
    # identifying entry points (OIS + 50 b.p. < 5Y swap rate)
    df = df[df.iloc[:,3] + 50*10**(-4) < df.iloc[:,8]]
    # computing weekly P&L of the fixed float carry
    df['Weekly P&L (USD)'] = df['Fixed 1-week Interest (USD)'] + df['Float 1-week Interest (USD)']
    # double checking interval
    df = df[df['Entry Date'] + timedelta(timedelta(days=365*interval).days) == df['Exit Date']]
    return df.dropna()

<h2> 2. Data </h2>

In [11]:
UK_OIS_1 = grab_quandl_table('YC/GBR_ISSC')['0.08Y']
UK_OIS_2 = grab_quandl_table('YC/GBR_ISSS', end_date = end_date)['0.08Y']
UK_OIS = pd.concat([UK_OIS_1,UK_OIS_2])/100

In [12]:
GBP_FX = grab_quandl_table('CUR/GBP', end_date = end_date).rename(columns={'RATE':"GBP"})

In [13]:
VNM_YC = grab_quandl_table('YC/VNM', end_date = end_date)/100
THA_YC = grab_quandl_table('YC/THA', end_date = end_date)/100
PAK_YC = grab_quandl_table('YC/PAK', end_date = end_date)/100
PHL_YC = grab_quandl_table('YC/PHL', end_date = end_date)/100

In [14]:
VND_FX = grab_quandl_table('CUR/VND', end_date = end_date).rename(columns={'RATE':"VND"})
THB_FX = grab_quandl_table('CUR/THB', end_date = end_date).rename(columns={'RATE':"THB"})
PKR_FX = grab_quandl_table('CUR/PKR', end_date = end_date).rename(columns={'RATE':"PKR"})
PHP_FX = grab_quandl_table('CUR/PHP', end_date = end_date).rename(columns={'RATE':"PHP"})

<h2> 3. Fixed-Float Carry </h2>

In [15]:
GBP_VND = FX_Fixed_Float_Carry(UK_OIS, GBP_FX, VNM_YC, VND_FX,
                               cap = weekly_investment_USD, interval = weekly_interval,
                               tenure = float_years, coupon_frequency = quarterly_coupon)

In [16]:
GBP_THB = FX_Fixed_Float_Carry(UK_OIS, GBP_FX, THA_YC, THB_FX,
                               cap = weekly_investment_USD, interval = weekly_interval,
                               tenure = float_years, coupon_frequency = quarterly_coupon)

In [17]:
GBP_PKR = FX_Fixed_Float_Carry(UK_OIS, GBP_FX, PAK_YC, PKR_FX,
                               cap = weekly_investment_USD, interval = weekly_interval,
                               tenure = float_years, coupon_frequency = quarterly_coupon)

In [18]:
GBP_PHP = FX_Fixed_Float_Carry(UK_OIS, GBP_FX, PHL_YC, PHP_FX,
                               cap = weekly_investment_USD, interval = weekly_interval,
                               tenure = float_years, coupon_frequency = quarterly_coupon)

<h2> 4. Analysis </h2>